In [1]:
# from __init__ import * 
from snmcseq_utils import create_logger
from snmcseq_utils import get_cluster_mc_c
# from CEMBA_update_mysql import connect_sql
# from CEMBA_init_ensemble_v2 import pull_genebody_info
# from CEMBA_init_ensemble_v2 import pull_binc_info

In [2]:
log = create_logger()

In [4]:
ens = 'Ens10'
context = 'CH'
df = get_cluster_mc_c(ens, context, genome_regions='genebody', cluster_type='mCHmCG_lv_npc50_k30', database='CEMBA')

03/19/2018 11:15:07 AM Output shape: (51772, 52)
